In [1]:
import wget
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
1

1

In [2]:
df=\
['/au/politics/articles?page=%s'%i for i in range(1,393)] +\
['/africa/politics/articles?page=%s'%i for i in range(1,63)] +\
['/ca/home-page/articles?page=%s'%i for i in range(1,157)] +\
['/global/home-page/articles?page=%s'%i for i in range(1,29)] +\
['/uk/politics/articles?page=%s'%i for i in range(1,339)] +\
['/us/politics/articles?page=%s'%i for i in range(1,124)]

df = pd.DataFrame(df, columns=['url'])
df.url  = 'https://theconversation.com' + df.url
df

In [3]:
def extract_rows(url) :
    output = []
    try:
        coverpage = requests.get(url).content
        soup = BeautifulSoup(coverpage, 'html.parser')
        rows = soup.find_all('div', class_ = 'article--header')  
        for row in rows:
            title = row.h2.text
            title = re.sub("^\s+|\s+$", "", title, flags=re.UNICODE) 
            href = row.h2.a['href']
            if href.startswith('/'):
                href = 'https://theconversation.com' + href
            output.append({'title':title, 'href':href})
    except:
        print('error', url)
    return output

In [4]:
df['rows']= None

In [5]:
pandarallel.initialize(nb_workers = 128)
df['rows'] = df['url'].parallel_apply(extract_rows).values

In [7]:
df

In [8]:
df[df.rows.isnull()]

In [13]:
df.loc[df.rows.str.len()==0, 'rows'] = df.loc[df.rows.str.len()==0, 'url'].parallel_apply(extract_rows).values

In [15]:
df[df.rows.isnull()]

,url,rows


In [16]:
df = df.explode('rows').reset_index(drop=True)
df

,url,rows
0,https://theconversation.com/au/politics/articles?page=1,"{'title': 'Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution', 'href': 'https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048'}"
1,https://theconversation.com/au/politics/articles?page=1,"{'title': 'History repeating: the surprising link between toxic masculinity and Australia’s convict past', 'href': 'https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881'}"
2,https://theconversation.com/au/politics/articles?page=1,"{'title': 'Morrison takes big personal hit in Newspoll after missteps on women’s issue', 'href': 'https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033'}"
3,https://theconversation.com/au/politics/articles?page=1,"{'title': 'VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House', 'href': 'https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969'}"
4,https://theconversation.com/au/politics/articles?page=1,"{'title': 'Men are more likely to commit violent crimes. Why is this so and how do we change it?', 'href': 'https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331'}"
...,...,...
27427,https://theconversation.com/us/politics/articles?page=123,"{'title': 'The tectonic plates of world Catholicism shift', 'href': 'https://theconversation.com/the-tectonic-plates-of-world-catholicism-shift-33066'}"
27428,https://theconversation.com/us/politics/articles?page=123,"{'title': 'How to make the House of Representatives representative', 'href': 'https://theconversation.com/how-to-make-the-house-of-representatives-representative-32921'}"
27429,https://theconversation.com/us/politics/articles?page=123,"{'title': 'Neither Rousseff nor Silva will do what’s right for Brazil', 'href': 'https://theconversation.com/neither-rousseff-nor-silva-will-do-whats-right-for-brazil-32351'}"
27430,https://theconversation.com/us/politics/articles?page=123,"{'title': 'The view from abroad on a year of Abbott’s muscular conservatism', 'href': 'https://theconversation.com/the-view-from-abroad-on-a-year-of-abbotts-muscular-conservatism-31765'}"


In [19]:
df1 = pd.json_normalize(df.rows)
df1

,title,href
0,Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution,https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048
1,History repeating: the surprising link between toxic masculinity and Australia’s convict past,https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881
2,Morrison takes big personal hit in Newspoll after missteps on women’s issue,https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033
3,VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House,https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969
4,Men are more likely to commit violent crimes. Why is this so and how do we change it?,https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331
...,...,...
27427,The tectonic plates of world Catholicism shift,https://theconversation.com/the-tectonic-plates-of-world-catholicism-shift-33066
27428,How to make the House of Representatives representative,https://theconversation.com/how-to-make-the-house-of-representatives-representative-32921
27429,Neither Rousseff nor Silva will do what’s right for Brazil,https://theconversation.com/neither-rousseff-nor-silva-will-do-whats-right-for-brazil-32351
27430,The view from abroad on a year of Abbott’s muscular conservatism,https://theconversation.com/the-view-from-abroad-on-a-year-of-abbotts-muscular-conservatism-31765


In [20]:
df1.to_csv('theconversation_raw.csv', index=False)

In [3]:
df1= pd.read_csv('theconversation_raw.csv', header=[0])
df1[['text','tags']]=None
df1

,title,href,text,tags
0,Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution,https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048,None,None
1,History repeating: the surprising link between toxic masculinity and Australia’s convict past,https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881,None,None
2,Morrison takes big personal hit in Newspoll after missteps on women’s issue,https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033,None,None
3,VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House,https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969,None,None
4,Men are more likely to commit violent crimes. Why is this so and how do we change it?,https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331,None,None
...,...,...,...,...
27427,The tectonic plates of world Catholicism shift,https://theconversation.com/the-tectonic-plates-of-world-catholicism-shift-33066,None,None
27428,How to make the House of Representatives representative,https://theconversation.com/how-to-make-the-house-of-representatives-representative-32921,None,None
27429,Neither Rousseff nor Silva will do what’s right for Brazil,https://theconversation.com/neither-rousseff-nor-silva-will-do-whats-right-for-brazil-32351,None,None
27430,The view from abroad on a year of Abbott’s muscular conservatism,https://theconversation.com/the-view-from-abroad-on-a-year-of-abbotts-muscular-conservatism-31765,None,None


In [3]:
from newspaper import Article, Config
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_text(url):
    try:
        article = Article(url,config= config)
        article.download()
        time.sleep(2)
        article.parse()
        text= article.text
        soup = BeautifulSoup(article.html, 'html.parser')
    except:
        print('error', url)
        return pd.Series([None]*2)
    try:
        tags = soup.find_all('li', class_= 'topic-list-item')
        tags = [i.a.text for i in tags]
    except:
        tags = None
    return pd.Series([text, tags])

In [9]:
pandarallel.initialize(nb_workers = 64)
batch_size = 64 * 8

i = resume = 0
while i< df1.shape[0]:
    print(i)
    df1.loc[i: i+ batch_size-1, ['text','tags']]=\
    df1.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_text).values
    i += batch_size
    if (i-resume)% (batch_size * 4) ==0:
        df1.to_csv('theconversation_text.csv', index= False)
df1.to_csv('theconversation_text.csv', index= False) 

1024
1536
2048
2560
3072
3584
4096
4608
5120
error https://theconversation.com/turkish-view-remains-neglected-in-our-understanding-of-gallipoli-38658
error https://theconversation.com/reviewing-an-anachronism-labor-to-debate-future-of-socialist-objective-45233error
 https://theconversation.com/can-the-real-malcolm-survive-being-pm-47928error
 https://theconversation.com/electronic-innovation-can-help-fix-an-archaic-crowded-prison-system-39044
error https://theconversation.com/what-kind-of-government-service-puts-public-on-hold-for-811-years-42314
error https://theconversation.com/seventy-years-after-hiroshima-who-was-australian-war-correspondent-wilfred-burchett-45965
error https://theconversation.com/as-morsi-faces-the-gallows-where-are-the-defenders-of-democracy-46068
error https://theconversation.com/would-australias-asylum-seeker-policy-stop-boats-to-europe-40645
error https://theconversation.com/caliphate-a-disputed-concept-no-longer-has-a-hold-over-all-muslims-41521
error https:/

In [4]:
df1= pd.read_csv('theconversation_text.csv', header=[0])
df1

,title,href,text,tags
0,Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution,https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048,"This week’s Newspoll, conducted March 24-27 from a sample of 1,517, gave Labor a 52-48 two party lead, unchanged from last fortnight’s Newspoll. Primary votes were 40% Coalition (up one), 38% Labor (down one), 11% Greens (up one) and 2% One Nation (down one).\n\nWhile voting intentions moved slightly towards the Coalition, Scott Morrison’s ratings fell to their lowest point since the COVID crisis began. 55% were satisfied with his performance (down seven) and 40% were dissatisfied (up six), for a net approval of +15, down 13 points.\n\nAnthony Albanese’s net approval was up one point to +2...","['Scott Morrison', 'redistribution', 'Newspoll', 'WA election']"
1,History repeating: the surprising link between toxic masculinity and Australia’s convict past,https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881,"It has been a momentous, demanding few weeks for Australia. Amid growing revelations of sexual assaults and toxic workplaces and people taking to the streets to voice their anger and frustration, it’s possible we are finally facing a reckoning on gender relations.\n\nBut as we debate — again — how to move into a more equal future, it is also useful to look to our convict past. This has an impact on the issues we face today, and in particular, our idea of masculine norms.\n\nMy research with colleagues Victoria Baranov and Ralph De Haas has used data from a unique natural experiment — convi...","['Mental health', 'Bullying', 'Violence', 'Gender equality', 'Masculinity', 'Sexual assault', 'Gender norms', 'Toxic masculinity', 'Census data']"
2,Morrison takes big personal hit in Newspoll after missteps on women’s issue,https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033,"Scott Morrison’s approval has taken a sizeable hit in a Newspoll showing Labor maintaining its 52-48% two-party lead.\n\nAs Morrison prepares to unveil his cabinet reshuffle the poll, published in Monday’s Australian, found his satisfaction rating fell from 62% to 55% in two weeks.\n\nThe fortnight was dominated by shocking revelations of lewd behaviour among staffers on the Coalition side, a botched attempted “reset” on gender issues when Morrison lashed out at a news conference, and a scandal around a Liberal backbencher.\n\nIn the poll, taken March 24-27, Morrison’s dissatisfaction rati...","['Scott Morrison', 'Newspoll', 'auspol', 'Anthony Albanese']"
3,VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House,https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969,"Michelle Grattan discusses the week in politics with University of Canberra Assistant Professor Caroline Fisher.\n\nThis week the pair discuss the latest in a growing list of allegations concerning sexual misconduct in Parliament House, and Scott Morrison’s response - a mea culpa, the possibility of quotas for women in the Liberal party, and a botched press conference. Also discussed is the likelihood of a cabinet reshuffle in light of the separate crisis’ involving Christian Porter and Linda Reynolds.","['Cabinet reshuffle', 'LNP', 'Scott Morrison', 'Christian Porter', 'Brittany Higgins']"
4,Men are more likely to commit violent crimes. Why is this so and how do we change it?,https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331,"Criminology is the study of individual and social factors associated with crime and the people who perpetrate it. One of the discipline’s well-established truths is that men commit violent and sexual offences at far higher rates than wo

In [13]:
# df1.to_csv('theconversation_text.csv', index= False)

In [5]:
df1[df1.text.isnull()]

,title,href,text,tags
5134,Can the ‘real Malcolm’ survive being PM?,https://theconversation.com/can-the-real-malcolm-survive-being-pm-47928,NaN,NaN
5135,Turnbull must break with past attempts to keep the future at bay,https://theconversation.com/turnbull-must-break-with-past-attempts-to-keep-the-future-at-bay-47687,NaN,NaN
5142,Turnbull unveils cabinet line-up: experts respond,https://theconversation.com/turnbull-unveils-cabinet-line-up-experts-respond-47626,NaN,NaN
5150,Iran: how a troubling actor could transform into a stabilising force,https://theconversation.com/iran-how-a-troubling-actor-could-transform-into-a-stabilising-force-46878,NaN,NaN
5158,Liberal spill: brutal business fells leaders who lack the skill set for the job at hand,https://theconversation.com/liberal-spill-brutal-business-fells-leaders-who-lack-the-skill-set-for-the-job-at-hand-47539,NaN,NaN
...,...,...,...,...
27131,Hillary Clinton stakes out safe political ground with energy and climate plan,https://theconversation.com/hillary-clinton-stakes-out-safe-political-ground-with-energy-and-climate-plan-45343,NaN,NaN
27132,Between a rock and a hard place: 2022 Winter Olympics decision,https://theconversation.com/between-a-rock-and-a-hard-place-2022-winter-olympics-decision-45219,NaN,NaN
27133,What is happening to civilians trapped in eastern Ukraine’s war zone?,https://theconversation.com/what-is-happening-to-civilians-trapped-in-eastern-ukraines-war-zone-45136,NaN,NaN
27134,What the Scott Walker fundraising controversy means for 2016,https://theconversation.com/what-the-scott-walker-fundraising-controversy-means-for-2016-45147,NaN,NaN


In [6]:
df2 = df1[df1.text.isnull()]
df2

,title,href,text,tags
5134,Can the ‘real Malcolm’ survive being PM?,https://theconversation.com/can-the-real-malcolm-survive-being-pm-47928,NaN,NaN
5135,Turnbull must break with past attempts to keep the future at bay,https://theconversation.com/turnbull-must-break-with-past-attempts-to-keep-the-future-at-bay-47687,NaN,NaN
5142,Turnbull unveils cabinet line-up: experts respond,https://theconversation.com/turnbull-unveils-cabinet-line-up-experts-respond-47626,NaN,NaN
5150,Iran: how a troubling actor could transform into a stabilising force,https://theconversation.com/iran-how-a-troubling-actor-could-transform-into-a-stabilising-force-46878,NaN,NaN
5158,Liberal spill: brutal business fells leaders who lack the skill set for the job at hand,https://theconversation.com/liberal-spill-brutal-business-fells-leaders-who-lack-the-skill-set-for-the-job-at-hand-47539,NaN,NaN
...,...,...,...,...
27131,Hillary Clinton stakes out safe political ground with energy and climate plan,https://theconversation.com/hillary-clinton-stakes-out-safe-political-ground-with-energy-and-climate-plan-45343,NaN,NaN
27132,Between a rock and a hard place: 2022 Winter Olympics decision,https://theconversation.com/between-a-rock-and-a-hard-place-2022-winter-olympics-decision-45219,NaN,NaN
27133,What is happening to civilians trapped in eastern Ukraine’s war zone?,https://theconversation.com/what-is-happening-to-civilians-trapped-in-eastern-ukraines-war-zone-45136,NaN,NaN
27134,What the Scott Walker fundraising controversy means for 2016,https://theconversation.com/what-the-scott-walker-fundraising-controversy-means-for-2016-45147,NaN,NaN


In [7]:
df2 = df2.reset_index(drop=True)

In [8]:
pandarallel.initialize(nb_workers = 20)
batch_size = 20 * 8

i = resume = 640
while i< df1.shape[0]:
    print(i)
    df2.loc[i: i+ batch_size-1, ['text','tags']]=\
    df2.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_text).values
    i += batch_size
    if (i-resume)% (batch_size * 4) ==0:
        df2.to_csv('df2.csv', index= False)
df2.to_csv('df2.csv', index= False) 

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
640
800
960
1120
1280
1440
1600
1760
1920
2080
2240
2400
2560
2720
2880
3040
3200
3360
3520
3680
3840
4000
4160
4320
4480
4640
4800
4960
5120
5280
5440
5600
5760
5920
6080
6240
6400
6560
6720
6880
7040
7200
7360
7520
7680
7840
8000
8160
8320
8480
8640
8800
8960
9120
9280
9440
9600
9760
9920
10080
10240
10400


ValueError: Number of processes must be at least 1

In [13]:
i = resume = 0
while i< 640:
    print(i)
    df2.loc[i: i+ batch_size-1, ['text','tags']]=\
    df2.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_text).values
    i += batch_size
    if (i-resume)% (batch_size * 4) ==0:
        df2.to_csv('df2.csv', index= False)
df2.to_csv('df2.csv', index= False) 

0
160
320
480


In [ ]:
quit()

In [3]:
df_link = pd.read_csv('theconversation_raw.csv', header=[0])
df_link

,title,href
0,Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution,https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048
1,History repeating: the surprising link between toxic masculinity and Australia’s convict past,https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881
2,Morrison takes big personal hit in Newspoll after missteps on women’s issue,https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033
3,VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House,https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969
4,Men are more likely to commit violent crimes. Why is this so and how do we change it?,https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331
...,...,...
27427,The tectonic plates of world Catholicism shift,https://theconversation.com/the-tectonic-plates-of-world-catholicism-shift-33066
27428,How to make the House of Representatives representative,https://theconversation.com/how-to-make-the-house-of-representatives-representative-32921
27429,Neither Rousseff nor Silva will do what’s right for Brazil,https://theconversation.com/neither-rousseff-nor-silva-will-do-whats-right-for-brazil-32351
27430,The view from abroad on a year of Abbott’s muscular conservatism,https://theconversation.com/the-view-from-abroad-on-a-year-of-abbotts-muscular-conservatism-31765


In [4]:
df1= pd.read_csv('theconversation_text.csv', header=[0])
df1

,title,href,text,tags
0,Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution,https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048,"This week’s Newspoll, conducted March 24-27 from a sample of 1,517, gave Labor a 52-48 two party lead, unchanged from last fortnight’s Newspoll. Primary votes were 40% Coalition (up one), 38% Labor (down one), 11% Greens (up one) and 2% One Nation (down one).\n\nWhile voting intentions moved slightly towards the Coalition, Scott Morrison’s ratings fell to their lowest point since the COVID crisis began. 55% were satisfied with his performance (down seven) and 40% were dissatisfied (up six), for a net approval of +15, down 13 points.\n\nAnthony Albanese’s net approval was up one point to +2...","['Scott Morrison', 'redistribution', 'Newspoll', 'WA election']"
1,History repeating: the surprising link between toxic masculinity and Australia’s convict past,https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881,"It has been a momentous, demanding few weeks for Australia. Amid growing revelations of sexual assaults and toxic workplaces and people taking to the streets to voice their anger and frustration, it’s possible we are finally facing a reckoning on gender relations.\n\nBut as we debate — again — how to move into a more equal future, it is also useful to look to our convict past. This has an impact on the issues we face today, and in particular, our idea of masculine norms.\n\nMy research with colleagues Victoria Baranov and Ralph De Haas has used data from a unique natural experiment — convi...","['Mental health', 'Bullying', 'Violence', 'Gender equality', 'Masculinity', 'Sexual assault', 'Gender norms', 'Toxic masculinity', 'Census data']"
2,Morrison takes big personal hit in Newspoll after missteps on women’s issue,https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033,"Scott Morrison’s approval has taken a sizeable hit in a Newspoll showing Labor maintaining its 52-48% two-party lead.\n\nAs Morrison prepares to unveil his cabinet reshuffle the poll, published in Monday’s Australian, found his satisfaction rating fell from 62% to 55% in two weeks.\n\nThe fortnight was dominated by shocking revelations of lewd behaviour among staffers on the Coalition side, a botched attempted “reset” on gender issues when Morrison lashed out at a news conference, and a scandal around a Liberal backbencher.\n\nIn the poll, taken March 24-27, Morrison’s dissatisfaction rati...","['Scott Morrison', 'Newspoll', 'auspol', 'Anthony Albanese']"
3,VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House,https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969,"Michelle Grattan discusses the week in politics with University of Canberra Assistant Professor Caroline Fisher.\n\nThis week the pair discuss the latest in a growing list of allegations concerning sexual misconduct in Parliament House, and Scott Morrison’s response - a mea culpa, the possibility of quotas for women in the Liberal party, and a botched press conference. Also discussed is the likelihood of a cabinet reshuffle in light of the separate crisis’ involving Christian Porter and Linda Reynolds.","['Cabinet reshuffle', 'LNP', 'Scott Morrison', 'Christian Porter', 'Brittany Higgins']"
4,Men are more likely to commit violent crimes. Why is this so and how do we change it?,https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331,"Criminology is the study of individual and social factors associated with crime and the people who perpetrate it. One of the discipline’s well-established truths is that men commit violent and sexual offences at far higher rates than wo

In [5]:
df1[df1.text.isnull()]

,title,href,text,tags
5134,Can the ‘real Malcolm’ survive being PM?,https://theconversation.com/can-the-real-malcolm-survive-being-pm-47928,NaN,NaN
5135,Turnbull must break with past attempts to keep the future at bay,https://theconversation.com/turnbull-must-break-with-past-attempts-to-keep-the-future-at-bay-47687,NaN,NaN
5142,Turnbull unveils cabinet line-up: experts respond,https://theconversation.com/turnbull-unveils-cabinet-line-up-experts-respond-47626,NaN,NaN
5150,Iran: how a troubling actor could transform into a stabilising force,https://theconversation.com/iran-how-a-troubling-actor-could-transform-into-a-stabilising-force-46878,NaN,NaN
5158,Liberal spill: brutal business fells leaders who lack the skill set for the job at hand,https://theconversation.com/liberal-spill-brutal-business-fells-leaders-who-lack-the-skill-set-for-the-job-at-hand-47539,NaN,NaN
...,...,...,...,...
27131,Hillary Clinton stakes out safe political ground with energy and climate plan,https://theconversation.com/hillary-clinton-stakes-out-safe-political-ground-with-energy-and-climate-plan-45343,NaN,NaN
27132,Between a rock and a hard place: 2022 Winter Olympics decision,https://theconversation.com/between-a-rock-and-a-hard-place-2022-winter-olympics-decision-45219,NaN,NaN
27133,What is happening to civilians trapped in eastern Ukraine’s war zone?,https://theconversation.com/what-is-happening-to-civilians-trapped-in-eastern-ukraines-war-zone-45136,NaN,NaN
27134,What the Scott Walker fundraising controversy means for 2016,https://theconversation.com/what-the-scott-walker-fundraising-controversy-means-for-2016-45147,NaN,NaN


In [6]:
df2= pd.read_csv('df2.csv', header=[0])
df2

,title,href,text,tags
0,Can the ‘real Malcolm’ survive being PM?,https://theconversation.com/can-the-real-malcolm-survive-being-pm-47928,"The late comedian Dave Allen once joked about then-British prime minister John Major that:\n\nIf John Major was drowning, his whole life would pass in front of him and he wouldn’t be in it.\n\nPolitical leadership is a funny thing. Allen’s joke is a reminder how easy it is for leaders to actually lose sight of who they are.\n\nIntelligent, ambitious and imaginative people fight and work and cajole their way to the top. They bend every faculty and characteristic to its best advantage as they convince those around them that they have what it takes to lead.\n\nEventually, they reach the apex ...","['Tony Abbott', 'Julia Gillard', 'Malcolm Turnbull', 'Political speech']"
1,Turnbull must break with past attempts to keep the future at bay,https://theconversation.com/turnbull-must-break-with-past-attempts-to-keep-the-future-at-bay-47687,"Australia’s new prime minister, Malcolm Turnbull, has announced what he calls a “21st-century government”. Today The Conversation begins a series focusing on what such a government should look like.\n\nWhen the history of the Abbott government is written, it will be well worth keeping the following example in mind.\n\nIn 1991, the Soviet Union and the old paradigm of communism teetered on the edge of extinction. Reacting to the seismic wave of freedom and people power sweeping the Soviet empire, a small group of hardline communist elites staged a coup.\n\nThey placed reformist president Mi...","['Tony Abbott', 'Governance', 'Malcolm Turnbull', 'Political history', 'Author Q&A', 'Political reform', 'Liberal leadership spill', '21st century government']"
2,Turnbull unveils cabinet line-up: experts respond,https://theconversation.com/turnbull-unveils-cabinet-line-up-experts-respond-47626,"Prime Minister Malcolm Turnbull has announced his frontbench line-up following last week’s leadership spill, which includes the promotion of Scott Morrison to treasurer among sweeping changes.\n\nSpeaking on Sunday, Turnbull said he was:\n\n… announcing a 21st-century government and a ministry for the future.\n\nGone from the frontbench entirely are Joe Hockey, Kevin Andrews, Eric Abetz, Bruce Billson and Ian Macfarlane. Josh Frydenberg (resources, energy and Northern Australia), Kelly O’Dwyer (small business and assistant treasurer), Marise Payne (defence), Mitch Fifield (communications a...","['Malcolm Turnbull', 'Cabinet', ""Kelly O'Dwyer"", 'Simon Birmingham', 'Liberal leadership spill']"
3,Iran: how a troubling actor could transform into a stabilising force,https://theconversation.com/iran-how-a-troubling-actor-could-transform-into-a-stabilising-force-46878,"This article is part of the Democracy Futures series, a joint global initiative with the Sydney Democracy Network. The project aims to stimulate fresh thinking about the many challenges facing democracies in the 21st century.\n\nDuring a historic visit to Tehran in 1977-78, US President Jimmy Carter said:\n\nIran is an island of stability in one of the more troubled areas of the world.\n\nThis was just one year before the 1979 revolution, which transformed Iran from a key US ally into the region’s most troubling actor. But today it is not unreasonable to claim that Iran’s ruling clergy has...","['Middle East', 'Iran', 'US foreign policy', 'Democracy Futures', 'Democracy Futures: Political Leadership', 'Iran nuclear deal']"
4,Liberal spill: brutal business fells leaders who lack the skill set for the job at hand,https://theconversation.com/liberal-spill-brutal-business-fells-leaders-who-lack-the-skill-set-for-the-job-at-hand-47539,"On November 19, 1990, the British prime minister, Margaret Thatcher, was in Paris at a European summit when she faced a press conference about the impending Conservative Party leadership spill back in London. She had gone to Paris knowing the challenge was on, happy to cast her vote by proxy. It wa

In [7]:
df2[df2.text.isnull()]

,title,href,text,tags


In [9]:
df1[df1.text.isnull()].href.isin(df2.href).shape

(10333,)

In [10]:
df_all = pd.concat([df1[~df1.text.isnull()],df2]).reset_index(drop=True)
df_all[df_all.duplicated(subset= 'href')]

,title,href,text,tags
200,"Men and women kill their children in roughly equal numbers, and we need to understand why",https://theconversation.com/men-and-women-kill-their-children-in-roughly-equal-numbers-and-we-need-to-understand-why-153527,"On average, one child is killed by a parent almost every fortnight in Australia.\n\nLast week, three children — Claire, 7, Anna, 5, and Matthew, 3 — were included in this terrible number. Homicide investigators have formed the “preliminary view” that their mother, Katie Perinovic, was responsible for their deaths before killing herself.\n\nTheir grieving father, Tomislav Perinovic, and Katie Perinovic’s parents have reportedly accepted the police’s version of events.\n\nIt was less than a year ago that Hannah Clarke and her children Aaliyah, Laianah and Trey were killed by their father, Ro...","['Domestic violence', 'Filicide', 'familicide', 'Hannah Clarke']"
400,Kumanjayi Walker murder trial will be a first in NT for an Indigenous death in custody. Why has it taken so long?,https://theconversation.com/kumanjayi-walker-murder-trial-will-be-a-first-in-nt-for-an-indigenous-death-in-custody-why-has-it-taken-so-long-148922,Aboriginal and Torres Strait Islander readers are advised this article contains names and images of deceased people.\n\nThe Northern Territory court decision this week to commit Police Constable Zachary Rolfe to stand trial for the shooting death of 19-year-old Warlpiri man Kumanjayi Walker marks a historic moment in accountability for First Nations deaths in custody.\n\nIt is the first time a police officer will face a murder trial in a First Nations death in custody case in the Northern Territory since the Royal Commission into Aboriginal Deaths in Custody in 1991.\n\nThe decision was ha...,"['Policing', 'northern territory', 'Deaths in custody', 'Royal Commission into Aboriginal Deaths in Custody', 'Indigenous deaths in custody', 'First Nations']"
650,Job recovery may be slowing as border closures have tightened: Frydenberg,https://theconversation.com/job-recovery-may-be-slowing-as-border-closures-have-tightened-frydenberg-144922,"More than half of the 1.3 million people who lost their jobs or were stood down on zero hours at the start of the pandemic had started some form of work by July, according to figures released by Treasurer Josh Frydenberg.\n\nTreasury figures show the national effective unemployment rate at 9.9% in July, compared with a peak of 14.9% in April, with 689,000 gaining effective employment. The official July unemployment rate was 7.5%.\n\nExcluding Victoria – which is in full lockdown combatting a second COVID wave - the national effective unemployment rate would be 9.5%.\n\nThe effective unempl...","['Unemployment', 'Josh Frydenberg']"
875,"Enforcing assimilation, dismantling Aboriginal families: a history of police violence in Australia",https://theconversation.com/enforcing-assimilation-dismantling-aboriginal-families-a-history-of-police-violence-in-australia-140637,"Readers are advised the following article contains descriptions of violence that may be traumatic.\n\nIn July 2018, Western Australia’s Police Commissioner Chris Dawson formally apologised for the mistreatment of Aboriginal people at the hands of police, acknowledging the “significant role” the police played in the dispossession of Australia’s First Nations people. Dawson made particular reference to the way:\n\nforceful removal of Aboriginal and Torres Strait Islander children from their families and communities, the displacement of mothers and their children, sisters, fathers and brother...","['Policing', 'Australian history', 'Aboriginal history', 'Frontier Wars', 'Black Lives Matter', 'Police violence', 'Aboriginal culture', 'Aboriginal people', 'Colonial rule', 'Colonial past', 'Aboriginal protest']"
1100,"The High Court rules in favour of News Corp, but against press freedom",https://theconversation.com/the-high-court-rules-in-favour-of-news-corp-but-against-press-freedom-136177

In [12]:
df_all.shape, df1.shape

((27432, 4), (27432, 4))

In [15]:
df_all = df_all[~df_all.duplicated(subset= ['href', 'text'])]

In [17]:
df_all[df_all.duplicated(subset= ['href'])]

,title,href,text,tags


In [30]:
df_all = df_all[df_all.text.str.len()>200]

In [27]:
df_all = df_all[~df_all.text.str.startswith('Want to write?')]
df_all = df_all[~df_all.text.str.startswith('Disclosure statement')]

In [31]:
df_all[df_all.duplicated(subset= ['text'])]

,title,href,text,tags


In [2]:
df= pd.read_csv('conversation.csv', header=[0])
df

,title,url,text,tags
0,Morrison’s ratings take a hit in Newspoll as Coalition notionally loses a seat in redistribution,https://theconversation.com/morrisons-ratings-take-a-hit-in-newspoll-as-coalition-notionally-loses-a-seat-in-redistribution-158048,"This week’s Newspoll, conducted March 24-27 from a sample of 1,517, gave Labor a 52-48 two party lead, unchanged from last fortnight’s Newspoll. Primary votes were 40% Coalition (up one), 38% Labor (down one), 11% Greens (up one) and 2% One Nation (down one).\n\nWhile voting intentions moved slightly towards the Coalition, Scott Morrison’s ratings fell to their lowest point since the COVID crisis began. 55% were satisfied with his performance (down seven) and 40% were dissatisfied (up six), for a net approval of +15, down 13 points.\n\nAnthony Albanese’s net approval was up one point to +2...","['Scott Morrison', 'redistribution', 'Newspoll', 'WA election']"
1,History repeating: the surprising link between toxic masculinity and Australia’s convict past,https://theconversation.com/history-repeating-the-surprising-link-between-toxic-masculinity-and-australias-convict-past-157881,"It has been a momentous, demanding few weeks for Australia. Amid growing revelations of sexual assaults and toxic workplaces and people taking to the streets to voice their anger and frustration, it’s possible we are finally facing a reckoning on gender relations.\n\nBut as we debate — again — how to move into a more equal future, it is also useful to look to our convict past. This has an impact on the issues we face today, and in particular, our idea of masculine norms.\n\nMy research with colleagues Victoria Baranov and Ralph De Haas has used data from a unique natural experiment — convi...","['Mental health', 'Bullying', 'Violence', 'Gender equality', 'Masculinity', 'Sexual assault', 'Gender norms', 'Toxic masculinity', 'Census data']"
2,Morrison takes big personal hit in Newspoll after missteps on women’s issue,https://theconversation.com/morrison-takes-big-personal-hit-in-newspoll-after-missteps-on-womens-issue-158033,"Scott Morrison’s approval has taken a sizeable hit in a Newspoll showing Labor maintaining its 52-48% two-party lead.\n\nAs Morrison prepares to unveil his cabinet reshuffle the poll, published in Monday’s Australian, found his satisfaction rating fell from 62% to 55% in two weeks.\n\nThe fortnight was dominated by shocking revelations of lewd behaviour among staffers on the Coalition side, a botched attempted “reset” on gender issues when Morrison lashed out at a news conference, and a scandal around a Liberal backbencher.\n\nIn the poll, taken March 24-27, Morrison’s dissatisfaction rati...","['Scott Morrison', 'Newspoll', 'auspol', 'Anthony Albanese']"
3,VIDEO: Michelle Grattan on Scott Morrison’s response to the culture of Parliament House,https://theconversation.com/video-michelle-grattan-on-scott-morrisons-response-to-the-culture-of-parliament-house-157969,"Michelle Grattan discusses the week in politics with University of Canberra Assistant Professor Caroline Fisher.\n\nThis week the pair discuss the latest in a growing list of allegations concerning sexual misconduct in Parliament House, and Scott Morrison’s response - a mea culpa, the possibility of quotas for women in the Liberal party, and a botched press conference. Also discussed is the likelihood of a cabinet reshuffle in light of the separate crisis’ involving Christian Porter and Linda Reynolds.","['Cabinet reshuffle', 'LNP', 'Scott Morrison', 'Christian Porter', 'Brittany Higgins']"
4,Men are more likely to commit violent crimes. Why is this so and how do we change it?,https://theconversation.com/men-are-more-likely-to-commit-violent-crimes-why-is-this-so-and-how-do-we-change-it-157331,"Criminology is the study of individual and social factors associated with crime and the people who perpetrate it. One of the discipline’s well-established truths is that men commit violent and sexual offences at far higher rates than wom